In [1]:
# Import needed modules
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import matplotlib.pylab as pylab

In [2]:
#Livin' in csv city
mjgross = pd.read_csv('D:/Flatiron/Project/Zips/bom.movie_gross.csv/bom.movie_gross.csv')
imdbname = pd.read_csv('D:/Flatiron/Project/Zips/imdb.name.basics.csv/name.basics.csv')
imdbtitle = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.akas.csv/title.akas.csv')
imdbtitle2 = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.basics.csv/title.basics.csv')
imdbcrew = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.crew.csv/title.crew.csv')
imdbprinc = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.principals.csv/title.principals.csv')
imdbrate = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.ratings.csv/title.ratings.csv')
rtinfo = pd.read_csv('D:/Flatiron/Project/Zips/rt.movie_info.tsv/rt.movie_info.tsv', delimiter='\t')
rtreviews = pd.read_csv('D:/Flatiron/Project/Zips/rt.reviews.tsv/rt.reviews.tsv', sep=None)
tmdbmovies = pd.read_csv('D:/Flatiron/Project/Zips/tmdb.movies.csv/tmdb.movies.csv')
tnbudgets = pd.read_csv('D:/Flatiron/Project/Zips/tn.movie_budgets.csv/tn.movie_budgets.csv')

In [3]:
# Make new dict of genres, cutting out fluff.
genre_dict = [{"id":28,"name":"Action"},{"id":12,"name":"Adventure"},{"id":16,"name":"Animation"},{"id":35,"name":"Comedy"},{"id":80,"name":"Crime"},{"id":99,"name":"Documentary"},{"id":18,"name":"Drama"},{"id":10751,"name":"Family"},{"id":14,"name":"Fantasy"},{"id":36,"name":"History"},{"id":27,"name":"Horror"},{"id":10402,"name":"Music"},{"id":9648,"name":"Mystery"},{"id":10749,"name":"Romance"},{"id":878,"name":"Science Fiction"},{"id":10770,"name":"TV Movie"},{"id":53,"name":"Thriller"},{"id":10752,"name":"War"},{"id":37,"name":"Western"}]
k = []
v = []
for i in range(len(genre_dict)):
    k.append(genre_dict[i]['id'])
    v.append(genre_dict[i]['name'])
gendic = dict(zip(k,v))
print(gendic)

{28: 'Action', 12: 'Adventure', 16: 'Animation', 35: 'Comedy', 80: 'Crime', 99: 'Documentary', 18: 'Drama', 10751: 'Family', 14: 'Fantasy', 36: 'History', 27: 'Horror', 10402: 'Music', 9648: 'Mystery', 10749: 'Romance', 878: 'Science Fiction', 10770: 'TV Movie', 53: 'Thriller', 10752: 'War', 37: 'Western'}


In [4]:
# Make lists out of Series.
titles = tmdbmovies['title'].to_list()
gennums = tmdbmovies['genre_ids'].to_list()

In [5]:
# Create empty DataFrame, using movie titles as index.
genresnp = np.zeros(shape=(26517, 19))
genre = pd.DataFrame(genresnp, columns = v)
genre = genre.applymap(lambda x: int(x))
genre['titles'] = titles
genre = genre.set_index('titles')

In [6]:
# Make list of lists of ints out of list of strings of lists of ints. It was as hard as it is confusing.
gennums = [i.strip("[]").split(", ") for i in gennums] 
gennums = [[int(j) for j in i if j] for i in gennums]

In [7]:
# Use list of int genres as keys to produce names of genres as columns in genre dictionary.
gennums = [[gendic[j] for j in i if j] for i in gennums]
gennums
n= -1
for i in gennums:
    n += 1
    for j in i:
        if j:
            genre.iloc[n].loc[j] += 1
genre
genres = genre.columns

In [8]:
######################## From here until specified I'm demonstrating all the work I put in before our conversation on Thursday. ######################## ########################
######################## My original plan when working with Huey's list was to compare critic and audience reviews to see what effect they have on a movie's gross.
######################## After talking to you and realizing I couldn't keep iterating on this small list of 21 movies and the point of the project was to work with large data sets, I threw most of this work away.
######################## I essentially started the project from scratch on Thursday, and went out of town Saturday and Sunday. I worked hard to give the presentation I did, even if it wasn't near complete.

# Huey's contribution to project.
df = pd.read_csv('D:/Flatiron/Project/Zips/bom.movie_gross.csv/bom.movie_gross.csv')
moneybank = []
for money in df.domestic_gross:
    if money > 400000000: # 400 mil and above
        moneybank.append(money)
top_dom_gross = {'Toy Story' : 415000000.0,
                "The Avengers": 623400000.0,
                 'The Dark Knight Rises' : 448100000.0,
                 'The Hunger Games' : 408000000.0,
                 'Frozen' : 400700000.0,
                 'Iron Man 3' : 409000000.0,
                 'The Hunger Games: Catching Fire' : 424700000.0,
                 'Star Wars: The Force Awakens' : 936700000.0,
                 'Jurassic World' : 652300000.0,
                 'Avengers: Age of Ultron' : 459000000.0,
                 'Captain America: Civil War' : 408100000.0,
                 'Rogue One: A Star Wars Story' : 532200000.0,
                 'Finding Dory' : 486300000.0,
                 'Star Wars: The Last Jedi' : 620200000.0,
                 'Beauty and the Beast' : 504000000.0,
                 'Jumanji: Welcome to the Jungle' : 404500000.0,
                 'Wonder Woman' : 412600000.0,
                 'Avengers: Infinity War' : 678800000.0,
                 'Black Panther' : 700100000.0,
                 'Jurassic World: Fallen Kingdom' :  417700000.0,
                 'Incredibles 2' : 608600000.0}
top21 = []
for movie in top_dom_gross:
    top21.append(movie)

In [9]:
# New df of genres of top 21 movies.
genre21 = genre[~genre.index.duplicated()]
genre21 = genre21.reindex(top21)

In [10]:
# Add columns for domestic gross and budget.
genre21 = genre[~genre.index.duplicated()]
genre21 = genre21.reindex(top21)
genre21['domgross'] = moneybank
tnbudgets = tnbudgets.set_index('movie')
tnbudgets21 = tnbudgets[~tnbudgets.index.duplicated()]
tnbudgets21 = tnbudgets21.reindex(top21)
budgets21 = list(tnbudgets21['production_budget'])
genre21['budget'] = budgets21
tnbudgets.reset_index()

,movie,id,release_date,production_budget,domestic_gross,worldwide_gross
0,Avatar,1,"Dec 18, 2009","$425,000,000","$760,507,625","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,2,"May 20, 2011","$410,600,000","$241,063,875","$1,045,663,875"
2,Dark Phoenix,3,"Jun 7, 2019","$350,000,000","$42,762,350","$149,762,350"
3,Avengers: Age of Ultron,4,"May 1, 2015","$330,600,000","$459,005,868","$1,403,013,963"
4,Star Wars Ep. VIII: The Last Jedi,5,"Dec 15, 2017","$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,Red 11,78,"Dec 31, 2018","$7,000",$0,$0
5778,Following,79,"Apr 2, 1999","$6,000","$48,482","$240,495"
5779,Return to the Land of Wonders,80,"Jul 13, 2005","$5,000","$1,338","$1,338"
5780,A Plague So Pleasant,81,"Sep 29, 2015","$1,400",$0,$0


In [11]:
# Clean budget column.
genre21['budget'] = genre21['budget'].str.replace(',','')
genre21['budget'] = genre21['budget'].fillna(10)
genre21['budget'] = genre21['budget'].apply(lambda x: int(str(x)[1:]))

In [12]:
# Found missing values using original source's website. Possible because of small data set.
genre21.loc['Star Wars: The Force Awakens', 'budget'] = 306000000
genre21.loc['Star Wars: The Last Jedi', 'budget'] = 200000000

In [13]:
# Use new columns to create informative columns.
genre21['return'] = genre21['domgross'] / genre21['budget'] 
genre21['profit'] = genre21['domgross'] - genre21['budget']

In [14]:
# Researched Rotten Tomatoes website, discovered url is based on movie title. Reformatted list to conform to url structure. 
top = []
for movie in top21:
    movie = movie.replace(' ', '_').replace(':', '').replace("'",'')
    top.append(movie)
# Manually fix entries that are named differently. Not sure how to automate.
top[14] = 'beauty_and_the_beast_2017'
top[7] = 'star_wars_episode_vii_the_force_awakens'
top[1] = 'marvels_the_avengers'
# Web Scrape RT to find critic and audience ratings for top 21 movies.
critic_ratings = []
audience_ratings = []
for i in range(21):
    url = 'http://www.rottentomatoes.com/m/{}'.format(top[i])
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    try:
        critic_ratings.append(soup.find('span', class_="mop-ratings-wrap__percentage").text.strip())
    except:
        critic_ratings.append(top[i])
    try:
        audience_ratings.append(soup.find('span', class_="mop-ratings-wrap__percentage").text.strip())
    except:
        audience_ratings.append(top[i])

In [15]:
# Add two new columns for ratings, clean and convert
genre21['critics'] = critic_ratings
genre21['audiences'] = audience_ratings
genre21['audiences'] = genre21['audiences'].apply(lambda x: int(x[0:-1]))
genre21['critics'] = genre21['critics'].apply(lambda x: int(x[0:-1]))
for c in genre21.columns:
    if c == 'return':
        continue
    genre21[c] = genre21[c].apply(lambda x: int(x))
genre21['return'] = genre21['return'].apply(lambda x: round(x, 1))
genre21

ValueError: invalid literal for int() with base 10: 'Toy_Stor'

In [16]:
# After our talk about using a larger data set, I changed plans.
# My first idea was to iterate through tmdbmovies to find genre ids that matched the top 3 genres by return. The return calculation is shown below. 
# tmdbmovies['hasTop3Genres'] = [True if all(elem in ast.literal_eval(g) for elem in top3) else False for g in tmdbmovies['genre_ids']]
# Very few movies met the all three genre criteria. Changed it to just animated movies as below. Problems listed at end.
animatedmovies = tmdbmovies[tmdbmovies['genre_ids'].str.contains('16')]
tmdbmovies['genre_ids'] = tmdbmovies['genre_ids'].apply(lambda x: ast.literal_eval(x))
anlist = list(animatedmovies.title)

# Tried to fix as many scenarios that would trip up the RT urls, which are based on titles. I imagine regex is the way to do this.
topan = []
for movie in anlist:
    movie = movie.replace(' ', '_').replace(':', '').replace("'",'').replace('-','_').replace('!','').replace('.','').replace('&','').replace('?','').replace('(','').replace(')','').replace('#','').replace('__','_').replace('__','_')
    topan.append(movie)

# Same process as before. List of animated movies was MUCH larger than the list of 21 movies above. This producued MANY more url errors (as shown by len(mistake)).
# As the only solution I knew to fix the urls was to do it by hand by searching through RT one title at a time, I found this extremely impractical.
# Threw away all this work. The successful run produced a list far too small to accurately finish project. Larger dataset produced too many errors.
critic_ratings = []
audience_ratings = []
mistake = []
ansuccess = []
for i in range(len(topan)):
    url = 'http://www.rottentomatoes.com/m/{}'.format(topan[i])
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    try:
        critic_ratings.append(soup.find('span', class_="mop-ratings-wrap__percentage").text.strip())
        ansuccess.append(topan[i])
    except:
        mistake.append(topan[i])
    try:
        audience_ratings.append(soup.find('span', class_="mop-ratings-wrap__percentage").text.strip())
    except:
        mistake.append(topan[i])
######################## Unapplicable work is over. Below is the project's work as presented. ######################## ########################

In [17]:
# Use data in tnbudgets to create a new column: return. Return on investment based on production budget vs domestic gross. Step one: lots of cleaning.
returnbudgets = tnbudgets
returnbudgets['production_budget'] = returnbudgets['production_budget'].str.replace(',','')
returnbudgets['production_budget'] = returnbudgets['production_budget'].fillna(10)
returnbudgets['production_budget'] = returnbudgets['production_budget'].apply(lambda x: int(str(x)[1:]))
returnbudgets['domestic_gross'] = returnbudgets['domestic_gross'].str.replace(',','')
returnbudgets['domestic_gross'] = returnbudgets['domestic_gross'].fillna(10)
returnbudgets['domestic_gross'] = returnbudgets['domestic_gross'].apply(lambda x: int(str(x)[1:]))
returnbudgets['return'] = returnbudgets['domestic_gross'] / returnbudgets['production_budget'] 

In [18]:
# Merge DataFrame with just genres with newly created DataFrame with budget,
genre = genre[~genre.index.duplicated()]
genre = genre[genre.notnull()]
genre = genre.reset_index()
returnbudgets = returnbudgets.reset_index()
genre = genre.merge(returnbudgets, left_on='titles', right_on='movie', how='inner')


In [34]:
# Break down movies by listed genre, assign that movie's return to a dictionary value of its genre, then divide by the number of movies adding values to the return to find average return by individual genre
# Sort genre by its values after switching keys and values. Could have altered code to skip the switching step
genrevalue = {}
for row in genre.index:
    genrecount = 0
    for g in genres:
        if genre.loc[row, g]:
            if g not in genrevalue:
                genrevalue[g] = genre.loc[row, 'return']
                genrecount += 1
            else:
                genrevalue[g] += genre.loc[row, 'return']
                genrecount += 1
        try:
            genrevalue[g] = int(genrevalue[g]) / genrecount
        except:
            continue
sortgenre = {k: v for k, v in sorted(genrevalue.items(), key=lambda item: item[1])}
sortgenre

{'Family': 0.0,
 'Fantasy': 0.0,
 'Science Fiction': 0.0,
 'Comedy': 0.0,
 'Thriller': 0.0,
 'Drama': 0.0,
 'Romance': 0.0,
 'Mystery': 0.0,
 'Crime': 0.0,
 'History': 0.0,
 'Western': 0.0,
 'War': 0.0,
 'Documentary': 0.0,
 'Music': 0.0,
 'TV Movie': 0.0,
 'Horror': 1.0,
 'Adventure': 2.0,
 'Animation': 22.0,
 'Action': 378.0}

In [ ]:
# Graph the genre results with a bar chart.
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (20, 10),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)
plt.xticks(rotation=30)
plt.ylabel('Return on Investment')
plt.bar(genrevalue.keys(), genrevalue.values());

In [ ]:
# Remove the overwhelmingly large result. Graph the rest.
sortgenrenoaction = sortgenre
del sortgenrenoaction['Action']
plt.xticks(rotation=30)
noaction = plt.bar(sortgenrenoaction.keys(), sortgenrenoaction.values())

In [ ]:
# Scatterplot with regression line. Budget vs gross.
ax = sns.lmplot(data=genre, x='production_budget', y='domestic_gross', size=10)
ax.set(xlabel='Budget (hundreds of millions)', ylabel= 'Domestic Gross (hundreds of millions');

In [17]:
# Same scatterplot with Animation genre highlighted
ax = sns.lmplot(data=genre, x='production_budget', y='domestic_gross', hue = 'Animation', legend_out=False, size=10)
new_labels = ['Human', 'Cartoon']
ax._legend.set_title(None)
for t, l in zip(ax._legend.texts, new_labels): t.set_text(l)
ax.set(xlabel='Budget, (hundreds of millions)', ylabel= 'Domestic Gross (hundreds of millions)');

KeyError: "['production_budget', 'domestic_gross'] not in index"